# Mortgage Risk

The example trains a model to perform home loan risk assessment using all of the loan data for the years 2000 to 2016 in the Fannie Mae loan performance dataset, consisting of roughly 400GB of data in memory.


<a id='contents'></a>
## Contents
__[Initialization](#initialize)__<br>
__[Pandas DataFrame](#pandas_dataframe)__<br>
>__[Model Training (CPU Only)](#pandas_model_training)__<br>
>__[Model Training (Multi-GPU)](#pandas_gpu_model_training)__<br>

__[cuDF DataFrame](#cudf_dataframe)__<br>
>__[Model Training (Multi-GPU)](#cudf_gpu_model_training)__<br>


<a id='initialize'></a>
## Initialize

The mortgage dataset used for this demo: __[Fannie Mae Loan Dataset](http://www.fanniemae.com/portal/funding-the-market/data/loan-performance-data.html)__


You can use these commands in a bash shell to download and decompress the 1TB dataset into the data folder.

In [ ]:
!nvidia-smi
#!nproc

In [ ]:
import numpy as np
import numba
import numba.cuda as cuda
import pandas as pd
import time
import xgboost as xgb
import cudf
import gc


In [ ]:
START_YEAR = 2000
END_YEAR = 2002 # end_year is not inclusive

ACQ_DATA_PATH = "/rapids/data/mortgage/acq"
PERF_DATA_PATH = "/rapids/data/mortgage/perf"
COL_NAMES_PATH = "/rapids/data/mortgage/names.csv"

PERCENT_TRAIN = 0.8
NUM_ROUNDS = 100

#The row stride to reduce the dataset for boosting
#1: Every row: 100% of the data
#2: Every 2nd row, 50% of the data
#3: Every 3rd row, 33% of the data
#etc
PART_COUNT = 1

<a id='pandas_dataframe'></a>
## Pandas Dataframe

### Data Preparation

In [ ]:
from collections import OrderedDict

def load_performance_csv(performance_path, **kwargs):
    """ Loads performance data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        "loan_id", "monthly_reporting_period", "servicer", "interest_rate", "current_actual_upb",
        "loan_age", "remaining_months_to_legal_maturity", "adj_remaining_months_to_maturity",
        "maturity_date", "msa", "current_loan_delinquency_status", "mod_flag", "zero_balance_code",
        "zero_balance_effective_date", "last_paid_installment_date", "foreclosed_after",
        "disposition_date", "foreclosure_costs", "prop_preservation_and_repair_costs",
        "asset_recovery_costs", "misc_holding_expenses", "holding_taxes", "net_sale_proceeds",
        "credit_enhancement_proceeds", "repurchase_make_whole_proceeds", "other_foreclosure_proceeds",
        "non_interest_bearing_upb", "principal_forgiveness_upb", "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount", "servicing_activity_indicator"
    ]
    
    #using categoricals to replace the string with a hashed int32
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("monthly_reporting_period", "date"),
        ("servicer", "category"),
        ("interest_rate", "float64"),
        ("current_actual_upb", "float64"),
        ("loan_age", "float64"),
        ("remaining_months_to_legal_maturity", "float64"),
        ("adj_remaining_months_to_maturity", "float64"),
        ("maturity_date", "date"),
        ("msa", "float64"),
        ("current_loan_delinquency_status", "int32"),
        ("mod_flag", "category"),
        ("zero_balance_code", "category"),
        ("zero_balance_effective_date", "date"),
        ("last_paid_installment_date", "date"),
        ("foreclosed_after", "date"),
        ("disposition_date", "date"),
        ("foreclosure_costs", "float64"),
        ("prop_preservation_and_repair_costs", "float64"),
        ("asset_recovery_costs", "float64"),
        ("misc_holding_expenses", "float64"),
        ("holding_taxes", "float64"),
        ("net_sale_proceeds", "float64"),
        ("credit_enhancement_proceeds", "float64"),
        ("repurchase_make_whole_proceeds", "float64"),
        ("other_foreclosure_proceeds", "float64"),
        ("non_interest_bearing_upb", "float64"),
        ("principal_forgiveness_upb", "float64"),
        ("repurchase_make_whole_proceeds_flag", "category"),
        ("foreclosure_principal_write_off_amount", "float64"),
        ("servicing_activity_indicator", "category")
    ])

    df = pd.read_csv(performance_path, names=cols, delimiter='|', skiprows=1)
    
    #dtype was not pass into read_csv, so convert/create categorical fields 
    for c in dtypes: 
        if (dtypes[c] == "category"):
            df[c] = pd.Categorical(df[c]).codes          
            
    return df  

def load_acquisition_csv(acquisition_path, **kwargs):
    """ Loads acquisition data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        'loan_id', 'orig_channel', 'seller_name', 'orig_interest_rate', 'orig_upb', 'orig_loan_term', 
        'orig_date', 'first_pay_date', 'orig_ltv', 'orig_cltv', 'num_borrowers', 'dti', 'borrower_credit_score', 
        'first_home_buyer', 'loan_purpose', 'property_type', 'num_units', 'occupancy_status', 'property_state',
        'zip', 'mortgage_insurance_percent', 'product_type', 'coborrow_credit_score', 'mortgage_insurance_type', 
        'relocation_mortgage_indicator'
    ]
 
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("orig_channel", "category"),
        ("seller_name", "category"),
        ("orig_interest_rate", "float64"),
        ("orig_upb", "int64"),
        ("orig_loan_term", "int64"),
        ("orig_date", "date"),
        ("first_pay_date", "date"),
        ("orig_ltv", "float64"),
        ("orig_cltv", "float64"),
        ("num_borrowers", "float64"),
        ("dti", "float64"),
        ("borrower_credit_score", "float64"),
        ("first_home_buyer", "category"),
        ("loan_purpose", "category"),
        ("property_type", "category"),
        ("num_units", "int64"),
        ("occupancy_status", "category"),
        ("property_state", "category"),
        ("zip", "int64"),
        ("mortgage_insurance_percent", "float64"),
        ("product_type", "category"),
        ("coborrow_credit_score", "float64"),
        ("mortgage_insurance_type", "float64"),
        ("relocation_mortgage_indicator", "category")
    ])
    
    #dtype was not pass into read_csv, so convert/create categorical fields 
    df = pd.read_csv(acquisition_path, names=cols, delimiter='|', skiprows=1)
    
    for c in dtypes: 
        if (dtypes[c] == "category"):
            df[c] = pd.Categorical(df[c]).codes          
    return df      

def load_names(col_names_path, **kwargs):
    """ Loads names used for renaming the banks
    
    Returns
    -------
    GPU DataFrame
    """

    cols = [
        'seller_name', 'new'
    ]

    #using categoricals to replace the string with a hashed int32
    dtypes = OrderedDict([
        ("seller_name", "category"),
        ("new", "category"),
    ])

    #dtype was not pass into read_csv, so convert/create categorical fields 
    df = pd.read_csv(col_names_path, names=cols, delimiter='|', skiprows=1)
    
    for c in dtypes: 
        if (dtypes[c] == "category"):
            df[c] = pd.Categorical(df[c]).codes          
       
    return df

ETL and Feature Engineering Functions

In [ ]:
pd.options.mode.chained_assignment = None  # default='warn'

def null_workaround(df, **kwargs):
    for column, data_type in df.dtypes.items():
        if str(data_type) == "category":
            df[column] = df[column].codes.astype('int32').fillna(-1)

        if str(data_type) in ['int8', 'int16', 'int32', 'int64', 'float32', 'float64']:
            df[column].fillna(-1, inplace=True)
    return df

def create_12_mon_features(joined_df, **kwargs):
    result_dfs = []
    n_months = 12
    for y in range(1, n_months + 1):
        temp_df = joined_df[['loan_id', 'timestamp_year', 'timestamp_month', 'delinquency_12', 'upb_12']]
        temp_df['temp_months'] = temp_df['timestamp_year'] * 12 + temp_df['timestamp_month']
        temp_df['temp_mody_n'] = np.floor((temp_df['temp_months'].astype('float64') - 24000 - y) / 12)
        temp_df = temp_df.groupby(['loan_id', 'temp_mody_n']).agg({'delinquency_12': 'max','upb_12': 'min'}).reset_index()
        temp_df.rename(columns={'delinquency_12':'max_delinquency_12','upb_12':'min_upb_12'}, inplace=True)
        
        temp_df['delinquency_12'] = (temp_df['max_delinquency_12']>3).astype('int32')
        temp_df['delinquency_12'] +=(temp_df['min_upb_12']==0).astype('int32')
        temp_df.drop('max_delinquency_12', axis=1, inplace=True)
        temp_df['upb_12'] = temp_df['min_upb_12']
        temp_df.drop('min_upb_12', axis=1, inplace=True)
        temp_df['timestamp_year'] = np.floor(((temp_df['temp_mody_n'] * n_months) + 24000 + (y - 1)) / 12).astype('int16')
        temp_df['timestamp_month'] = np.int8(y)
        temp_df.drop('temp_mody_n', axis=1, inplace=True)
        
        result_dfs.append(temp_df)
        del(temp_df)
        
    del(joined_df)
    return pd.concat(result_dfs)

def create_ever_features(df, **kwargs):
    ever_df = df[['loan_id', 'current_loan_delinquency_status']]
    ever_df = ever_df.groupby('loan_id').max().reset_index()
    ever_df.rename(columns={'current_loan_delinquency_status':'max_current_loan_delinquency_status'}, inplace=True)
    del(df)
    
    ever_df['ever_30'] = (ever_df['max_current_loan_delinquency_status'] >= 1).astype('int8')
    ever_df['ever_90'] = (ever_df['max_current_loan_delinquency_status'] >= 3).astype('int8')
    ever_df['ever_180'] = (ever_df['max_current_loan_delinquency_status'] >= 6).astype('int8')
    ever_df.drop('max_current_loan_delinquency_status', axis=1, inplace=True, errors = 'ignore')
    return ever_df

def create_delinq_features(df, **kwargs):
    delinq_df = df[['loan_id', 'monthly_reporting_period', 'current_loan_delinquency_status']]
    del(df)
    
    delinq_30 = delinq_df.query('current_loan_delinquency_status >= 1')[['loan_id', 'monthly_reporting_period']].groupby('loan_id').min().reset_index()
    delinq_30.rename(columns={'monthly_reporting_period':'min_monthly_reporting_period'}, inplace=True)
    delinq_30['delinquency_30'] = delinq_30['min_monthly_reporting_period']
    delinq_30.drop('min_monthly_reporting_period', axis=1, inplace=True)
    
    delinq_90 = delinq_df.query('current_loan_delinquency_status >= 3')[['loan_id', 'monthly_reporting_period']].groupby('loan_id').min().reset_index()
    delinq_90.rename(columns={'monthly_reporting_period':'min_monthly_reporting_period'}, inplace=True)
    delinq_90['delinquency_90'] = delinq_90['min_monthly_reporting_period']
    delinq_90.drop('min_monthly_reporting_period', axis=1, inplace=True)
    
    delinq_180 = delinq_df.query('current_loan_delinquency_status >= 6')[['loan_id', 'monthly_reporting_period']].groupby('loan_id').min().reset_index()
    delinq_180.rename(columns={'monthly_reporting_period':'min_monthly_reporting_period'}, inplace=True)
    delinq_180['delinquency_180'] = delinq_180['min_monthly_reporting_period']
    delinq_180.drop('min_monthly_reporting_period', axis=1, inplace=True)
    del(delinq_df)
    
    delinq_merge_df = delinq_30.merge(delinq_90, how='left', on=['loan_id'])
    delinq_merge_df['delinquency_90'] = delinq_merge_df['delinquency_90'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    delinq_merge_df = delinq_merge_df.merge(delinq_180, how='left', on=['loan_id'])
    delinq_merge_df['delinquency_180'] = delinq_merge_df['delinquency_180'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    del(delinq_30)
    del(delinq_90)
    del(delinq_180)
    return delinq_merge_df

def join_ever_delinq_features(df, delinq_df, **kwargs):
    ever_df = df.merge(delinq_df, on=['loan_id'], how='left')
    del(df)
    del(delinq_df)
    
    ever_df['delinquency_30'] = ever_df['delinquency_30'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    ever_df['delinquency_90'] = ever_df['delinquency_90'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    ever_df['delinquency_180'] = ever_df['delinquency_180'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    return ever_df

def create_joined_df(df, ever_df, **kwargs):
    test = df[['loan_id', 'monthly_reporting_period', 'current_loan_delinquency_status', 'current_actual_upb']]
    del(df)
    
    test['timestamp'] = pd.to_datetime(test['monthly_reporting_period'], infer_datetime_format=True)
    test.drop('monthly_reporting_period', axis=1, inplace=True)
    test['timestamp_month'] = test['timestamp'].dt.month
    test['timestamp_year'] = test['timestamp'].dt.year
    test['delinquency_12'] = test['current_loan_delinquency_status']
    test.drop('current_loan_delinquency_status', axis=1, inplace=True)
    test['upb_12'] = test['current_actual_upb']
    test.drop('current_actual_upb', axis=1, inplace=True)
    test['upb_12'].fillna(999999999, inplace=True)
    test['delinquency_12'].fillna(-1, inplace=True)
    
    joined_df = test.merge(ever_df, how='left', on=['loan_id'])
    del(ever_df)
    del(test)
    
    joined_df['ever_30'].fillna(-1, inplace=True)
    joined_df['ever_90'].fillna(-1, inplace=True)
    joined_df['ever_180'].fillna(-1, inplace=True)
    joined_df['delinquency_30'].fillna(-1, inplace=True)
    joined_df['delinquency_90'].fillna(-1, inplace=True)
    joined_df['delinquency_180'].fillna(-1, inplace=True)
    
    joined_df['timestamp_year'] = joined_df['timestamp_year'].astype('int32')
    joined_df['timestamp_month'] = joined_df['timestamp_month'].astype('int32')
    
    return joined_df

def combine_joined_12_mon(joined_df, testdf, **kwargs):
    joined_df.drop('delinquency_12', axis=1, inplace=True)
    joined_df.drop('upb_12', axis=1, inplace=True)
    joined_df['timestamp_year'] = joined_df['timestamp_year'].astype('int16')
    joined_df['timestamp_month'] = joined_df['timestamp_month'].astype('int8')
    return joined_df.merge(testdf, how='left', on=['loan_id', 'timestamp_year', 'timestamp_month'])

def final_performance_delinquency(df, joined_df, **kwargs):
    merged_df = null_workaround(df)
    joined_df = null_workaround(joined_df)
    merged_df['timestamp_month'] = pd.to_datetime(merged_df['monthly_reporting_period'], infer_datetime_format=True).dt.month
    merged_df['timestamp_month'] = merged_df['timestamp_month'].astype('int8')
    merged_df['timestamp_year'] = pd.to_datetime(merged_df['monthly_reporting_period'], infer_datetime_format=True).dt.year
    merged_df['timestamp_year'] = merged_df['timestamp_year'].astype('int16')
    merged_df = merged_df.merge(joined_df, how='left', on=['loan_id', 'timestamp_year', 'timestamp_month'])
    merged_df.drop('timestamp_year', axis=1, inplace=True)
    merged_df.drop('timestamp_month', axis=1, inplace=True)
    return merged_df

def join_perf_acq_gdfs(perf_df, acq_df, **kwargs):
    perf_df = null_workaround(perf_df)
    acq_df = null_workaround(acq_df)
    return perf_df.merge(acq_df, how='left', on=['loan_id'])

def last_mile_cleaning(df, **kwargs):
    drop_list = [
        'loan_id', 'orig_date', 'first_pay_date', 'seller_name',
        'monthly_reporting_period', 'last_paid_installment_date', 'maturity_date', 'ever_30', 'ever_90', 'ever_180',
        'delinquency_30', 'delinquency_90', 'delinquency_180', 'upb_12',
        'zero_balance_effective_date','foreclosed_after', 'disposition_date','timestamp'
    ]
    #for column in drop_list:
    df.drop(drop_list, axis=1, inplace=True)
    
    for col, dtype in df.dtypes.iteritems():
        if str(dtype)=='category':
            df[col] = df[col].cat.codes
        df[col] = df[col].astype('float32')
    df['delinquency_12'] = df['delinquency_12'] > 0
    df['delinquency_12'] = df['delinquency_12'].fillna(False).astype('int32')
    return df

In [ ]:
import glob
from glob import glob
import os
import pandas as pd

def process_quarter(names_df, acq_df, perf_df):
    acq_df = acq_df.merge(names_df, how='left', on=['seller_name'])
    acq_df.drop('seller_name', axis=1, inplace=True)
    acq_df['seller_name'] = acq_df['new']
    acq_df.drop('new', axis=1, inplace=True)
        
    df = perf_df
    ever_df = create_ever_features(df)
    delinq_merge = create_delinq_features(df)
    ever_df = join_ever_delinq_features(ever_df, delinq_merge)
    del(delinq_merge)
 
    joined_df = create_joined_df(df, ever_df)
    test_df = create_12_mon_features(joined_df)
    joined_df = combine_joined_12_mon(joined_df, test_df)
    del(test_df)
    
    perf_df = final_performance_delinquency(df, joined_df)
    del(df, joined_df)
    
    final_df = join_perf_acq_gdfs(perf_df, acq_df)
    del(perf_df)
    del(acq_df)
  
    final_df = last_mile_cleaning(final_df)
    return final_df

def process_quarter_file(acquisition_file, perf_file):
    print("Processing file: {0}".format(perf_file))
    
    #Load Data
    names_df = load_names(COL_NAMES_PATH)
    acq_df = load_acquisition_csv(acquisition_file)
    perf_df = load_performance_csv(perf_file)
    
    df = process_quarter(names_df, acq_df, perf_df)
    del(acq_df, perf_df, names_df)
    
    return df


In [ ]:
%%time

#read in data files onr at a time because pandas performance does not improve with multi-threading
frames = []
quarter = 1
year = START_YEAR
while year != END_YEAR:
    acquisition_file = ACQ_DATA_PATH + "/Acquisition_" + str(year) + "Q" + str(quarter) + ".txt"
    files = glob(os.path.join(PERF_DATA_PATH + "/Performance_" + str(year) + "Q" + str(quarter) + "*"))
    for file in files:
        df1 = process_quarter_file(acquisition_file, file)
        frames.append(df1) 

    quarter += 1
    if quarter == 5:
        year += 1
        quarter = 1
        
print("Concatenating dataframes")
df_train = pd.concat(frames)    
del frames

#Optional: Review the data and data types
#print (df_train.head())
#print(df_train.dtypes)

In [ ]:
#filter the data by the desired columns

PRED_VARS_X = df_train.columns.difference(['delinquency_12'])
PRED_VARS_Y ='delinquency_12'

x = df_train[PRED_VARS_X]
y = df_train[PRED_VARS_Y]

In [ ]:
rows_train = int(int(len(df_train)) * PERCENT_TRAIN)
print(rows_train)

In [ ]:
%%time 

x_train = x[:rows_train:PART_COUNT]
x_test = x[rows_train::PART_COUNT]
y_train = y[:rows_train:PART_COUNT]
y_test = y[rows_train::PART_COUNT]


<a id='pandas_model_training'></a>
### Model Training
CPU XGBoost using the *fit* method

In [ ]:
import multiprocessing

#Optional: loop over CPU threads for sizing and performance
NTHREADS=multiprocessing.cpu_count()

print("threads: {0}".format(NTHREADS))

In [ ]:
%%time

model = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators = NUM_ROUNDS,
    gamma = 1,
    eta=0.1,
    alpha=0.9,
    subsample=1,
    reg_lambda = 1,
    scale_pos_weight = 2,
    min_child_weight = 30,    
    max_depth = 8,
    early_stopping_rounds = 5,
    max_features = 'auto',
    max_leaves = 2**8,
    objective = 'reg:linear',
    booster = 'gbtree',
    criterion = 'friedman_mse',
    grow_policy = 'lossguide',
    tree_method = 'hist',
    nthread = NTHREADS,
    silent = True)

gbm = model.fit(x_train, y_train)

### Evaluation
Accuracy (AUC) is measured by the area under the ROC curve. The definition of an acceptable AUC is relative and not absolute based on the data and feature. An area of 1 represents a perfect test; an area of .5 represents a poor or unrealiable test. A suggested rough guide for classifying the accuracy of a XGBoost:<br>
>90 - 100 = Excellent <br>
>80 - 90 = Good <br>
>70 - 80 = Fair <br>
>60 - 70 = Poor <br>
>50 - 60 = Fail <br>
>50 and Below = Unreliable

In [ ]:
from sklearn import metrics

pred = gbm.predict(x_test)

fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
auc = metrics.auc(fpr, tpr) * 100

acc = metrics.roc_auc_score(y_test, pred) * 100

print("AUC: {0}, ROC AUC: {1}".format(auc, acc))

In [ ]:
#cleanup for the next iteration
del fpr
del tpr
del thresholds 
del acc
del auc
del pred

del model
del gbm

gc.collect()

For additional iterations, goto top of [model training section](#pandas_model_training).

<a id='pandas_gpu_model_training'></a>
### Model Training
GPU accelerated XGBoost using the *fit* method

In [ ]:
#To use all GPUs, set NGPUS=-1
#Optional: loop over number of GPUs for sizing and performance
NGPUS=-1

In [ ]:
%%time

model = xgb.XGBClassifier(
    learning_rate = 0.1,
    n_estimators = NUM_ROUNDS,
    gamma = 1,
    eta=0.1,
    alpha=0.9,
    subsample=1,
    reg_lambda = 1,
    scale_pos_weight = 2,
    min_child_weight = 30,    
    max_depth = 8,
    early_stopping_rounds = 5,
    max_features = 'auto',
    max_leaves = 2**8,
    objective = 'gpu:reg:linear',
    booster = 'gbtree',
    criterion = 'friedman_mse',
    grow_policy = 'lossguide',
    tree_method = 'gpu_hist',
    n_gpus = NGPUS,
    silent = True)

gbm = model.fit(x_train, y_train)

### Evaluation
Accuracy (AUC) is measured by the area under the ROC curve. The definition of an acceptable AUC is relative and not absolute based on the data and feature. An area of 1 represents a perfect test; an area of .5 represents a poor or unrealiable test. A suggested rough guide for classifying the accuracy of a XGBoost:<br>
>90 - 100 = Excellent <br>
>80 - 90 = Good <br>
>70 - 80 = Fair <br>
>60 - 70 = Poor <br>
>50 - 60 = Fail <br>
>50 and Below = Unreliable

In [ ]:
from sklearn import metrics

pred = gbm.predict(x_test)

fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
auc = metrics.auc(fpr, tpr) * 100

acc = metrics.roc_auc_score(y_test, pred) * 100

print("AUC: {0}, ROC AUC: {1}".format(auc, acc))

In [ ]:
#cleanup for the next iteration
del fpr
del tpr
del thresholds 
del acc
del auc
del pred

del model
del gbm

gc.collect()

For additional iterations, goto top of [model training section](#pandas_gpu_model_training).

In [ ]:
#After looping through the thead counts, cleanup for the benchmark
del x
del y

del x_train
del x_test 
del y_train 
del y_test

del df_train
#del df_validate

gc.collect()

<a id='cudf_dataframe'></a>
## cuDF DataFrame

### Data Preparation

In [ ]:
from collections import OrderedDict

def cudf_load_performance_csv(performance_path, **kwargs):
    """ Loads performance data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        "loan_id", "monthly_reporting_period", "servicer", "interest_rate", "current_actual_upb",
        "loan_age", "remaining_months_to_legal_maturity", "adj_remaining_months_to_maturity",
        "maturity_date", "msa", "current_loan_delinquency_status", "mod_flag", "zero_balance_code",
        "zero_balance_effective_date", "last_paid_installment_date", "foreclosed_after",
        "disposition_date", "foreclosure_costs", "prop_preservation_and_repair_costs",
        "asset_recovery_costs", "misc_holding_expenses", "holding_taxes", "net_sale_proceeds",
        "credit_enhancement_proceeds", "repurchase_make_whole_proceeds", "other_foreclosure_proceeds",
        "non_interest_bearing_upb", "principal_forgiveness_upb", "repurchase_make_whole_proceeds_flag",
        "foreclosure_principal_write_off_amount", "servicing_activity_indicator"
    ]
    
    #using categoricals to replace the string with a hashed int32
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("monthly_reporting_period", "date"),
        ("servicer", "category"),
        ("interest_rate", "float64"),
        ("current_actual_upb", "float64"),
        ("loan_age", "float64"),
        ("remaining_months_to_legal_maturity", "float64"),
        ("adj_remaining_months_to_maturity", "float64"),
        ("maturity_date", "date"),
        ("msa", "float64"),
        ("current_loan_delinquency_status", "int32"),
        ("mod_flag", "category"),
        ("zero_balance_code", "category"),
        ("zero_balance_effective_date", "date"),
        ("last_paid_installment_date", "date"),
        ("foreclosed_after", "date"),
        ("disposition_date", "date"),
        ("foreclosure_costs", "float64"),
        ("prop_preservation_and_repair_costs", "float64"),
        ("asset_recovery_costs", "float64"),
        ("misc_holding_expenses", "float64"),
        ("holding_taxes", "float64"),
        ("net_sale_proceeds", "float64"),
        ("credit_enhancement_proceeds", "float64"),
        ("repurchase_make_whole_proceeds", "float64"),
        ("other_foreclosure_proceeds", "float64"),
        ("non_interest_bearing_upb", "float64"),
        ("principal_forgiveness_upb", "float64"),
        ("repurchase_make_whole_proceeds_flag", "category"),
        ("foreclosure_principal_write_off_amount", "float64"),
        ("servicing_activity_indicator", "category")
    ])

    return cudf.read_csv(performance_path, names=cols, delimiter='|', dtype=list(dtypes.values()), skiprows=1)

def cudf_load_acquisition_csv(acquisition_path, **kwargs):
    """ Loads acquisition data

    Returns
    -------
    GPU DataFrame
    """
    
    cols = [
        'loan_id', 'orig_channel', 'seller_name', 'orig_interest_rate', 'orig_upb', 'orig_loan_term', 
        'orig_date', 'first_pay_date', 'orig_ltv', 'orig_cltv', 'num_borrowers', 'dti', 'borrower_credit_score', 
        'first_home_buyer', 'loan_purpose', 'property_type', 'num_units', 'occupancy_status', 'property_state',
        'zip', 'mortgage_insurance_percent', 'product_type', 'coborrow_credit_score', 'mortgage_insurance_type', 
        'relocation_mortgage_indicator'
    ]
 
    #using categoricals to replace the string with a hashed int32
    dtypes = OrderedDict([
        ("loan_id", "int64"),
        ("orig_channel", "category"),
        ("seller_name", "category"),
        ("orig_interest_rate", "float64"),
        ("orig_upb", "int64"),
        ("orig_loan_term", "int64"),
        ("orig_date", "date"),
        ("first_pay_date", "date"),
        ("orig_ltv", "float64"),
        ("orig_cltv", "float64"),
        ("num_borrowers", "float64"),
        ("dti", "float64"),
        ("borrower_credit_score", "float64"),
        ("first_home_buyer", "category"),
        ("loan_purpose", "category"),
        ("property_type", "category"),
        ("num_units", "int64"),
        ("occupancy_status", "category"),
        ("property_state", "category"),
        ("zip", "int64"),
        ("mortgage_insurance_percent", "float64"),
        ("product_type", "category"),
        ("coborrow_credit_score", "float64"),
        ("mortgage_insurance_type", "float64"),
        ("relocation_mortgage_indicator", "category")
    ])
    
    return cudf.read_csv(acquisition_path, names=cols, delimiter='|', dtype=list(dtypes.values()), skiprows=1)

def cudf_load_names(col_names_path, **kwargs):
    """ Loads names used for renaming the banks
    
    Returns
    -------
    GPU DataFrame
    """

    cols = [
        'seller_name', 'new'
    ]

    #using categoricals to replace the string with a hashed int32
    dtypes = OrderedDict([
        ("seller_name", "category"),
        ("new", "category"),
    ])

    return cudf.read_csv(col_names_path, names=cols, delimiter='|', dtype=list(dtypes.values()), skiprows=1)

ETL and Feature Engineering Functions

In [ ]:

def cudf_null_workaround(df, **kwargs):
    for column, data_type in df.dtypes.items():
        if str(data_type) == "category":
            df[column] = df[column].astype('int32').fillna(-1)
        if str(data_type) in ['int8', 'int16', 'int32', 'int64', 'float32', 'float64']:
            df[column] = df[column].fillna(-1)
    return df

def cudf_create_12_mon_features(joined_df, **kwargs):
    result_dfs = []
    n_months = 12
    for y in range(1, n_months + 1):
        temp_df = joined_df[['loan_id', 'timestamp_year', 'timestamp_month', 'delinquency_12', 'upb_12']]
        temp_df['temp_months'] = temp_df['timestamp_year'] * 12 + temp_df['timestamp_month']
        temp_df['temp_mody_n'] = ((temp_df['temp_months'].astype('float64') - 24000 - y) / 12).floor()
        temp_df = temp_df.groupby(['loan_id', 'temp_mody_n'], method='hash').agg({'delinquency_12': 'max','upb_12': 'min'})
        temp_df['delinquency_12'] = (temp_df['max_delinquency_12']>3).astype('int32')
        temp_df['delinquency_12'] +=(temp_df['min_upb_12']==0).astype('int32')
        temp_df.drop_column('max_delinquency_12')
        temp_df['upb_12'] = temp_df['min_upb_12']
        temp_df.drop_column('min_upb_12')
        temp_df['timestamp_year'] = (((temp_df['temp_mody_n'] * n_months) + 24000 + (y - 1)) / 12).floor().astype('int16')
        temp_df['timestamp_month'] = np.int8(y)
        temp_df.drop_column('temp_mody_n')
        
        result_dfs.append(temp_df)
        del(temp_df)
        
    del(joined_df)
    return cudf.concat(result_dfs)

def cudf_create_ever_features(df, **kwargs):
    ever_df = df[['loan_id', 'current_loan_delinquency_status']]
    ever_df = ever_df.groupby('loan_id', method='hash').max()
    del(df)
    
    ever_df['ever_30'] = (ever_df['max_current_loan_delinquency_status'] >= 1).astype('int8')
    ever_df['ever_90'] = (ever_df['max_current_loan_delinquency_status'] >= 3).astype('int8')
    ever_df['ever_180'] = (ever_df['max_current_loan_delinquency_status'] >= 6).astype('int8')
    ever_df.drop_column('max_current_loan_delinquency_status')
    return ever_df

def cudf_create_delinq_features(df, **kwargs):
    delinq_df = df[['loan_id', 'monthly_reporting_period', 'current_loan_delinquency_status']]
    del(df)
    
    delinq_30 = delinq_df.query('current_loan_delinquency_status >= 1')[['loan_id', 'monthly_reporting_period']].groupby('loan_id', method='hash').min()
    delinq_30['delinquency_30'] = delinq_30['min_monthly_reporting_period']
    delinq_30.drop_column('min_monthly_reporting_period')
    delinq_90 = delinq_df.query('current_loan_delinquency_status >= 3')[['loan_id', 'monthly_reporting_period']].groupby('loan_id', method='hash').min()
    delinq_90['delinquency_90'] = delinq_90['min_monthly_reporting_period']
    delinq_90.drop_column('min_monthly_reporting_period')
    delinq_180 = delinq_df.query('current_loan_delinquency_status >= 6')[['loan_id', 'monthly_reporting_period']].groupby('loan_id', method='hash').min()
    delinq_180['delinquency_180'] = delinq_180['min_monthly_reporting_period']
    delinq_180.drop_column('min_monthly_reporting_period')
    del(delinq_df)
    
    delinq_merge_df = delinq_30.merge(delinq_90, how='left', on=['loan_id'], type='hash')
    delinq_merge_df['delinquency_90'] = delinq_merge_df['delinquency_90'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    delinq_merge_df = delinq_merge_df.merge(delinq_180, how='left', on=['loan_id'], type='hash')
    delinq_merge_df['delinquency_180'] = delinq_merge_df['delinquency_180'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    del(delinq_30)
    del(delinq_90)
    del(delinq_180)
    return delinq_merge_df

def cudf_join_ever_delinq_features(df, delinq_df, **kwargs):
    ever_df = df.merge(delinq_df, on=['loan_id'], how='left', type='hash')
    del(df)
    del(delinq_df)
    
    ever_df['delinquency_30'] = ever_df['delinquency_30'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    ever_df['delinquency_90'] = ever_df['delinquency_90'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    ever_df['delinquency_180'] = ever_df['delinquency_180'].fillna(np.dtype('datetime64[ms]').type('1970-01-01').astype('datetime64[ms]'))
    return ever_df

def cudf_create_joined_df(df, ever_df, **kwargs):
    test = df[['loan_id', 'monthly_reporting_period', 'current_loan_delinquency_status', 'current_actual_upb']]
    del(df)
    
    test['timestamp'] = test['monthly_reporting_period']
    test.drop_column('monthly_reporting_period')
    test['timestamp_month'] = test['timestamp'].dt.month
    test['timestamp_year'] = test['timestamp'].dt.year
    test['delinquency_12'] = test['current_loan_delinquency_status']
    test.drop_column('current_loan_delinquency_status')
    test['upb_12'] = test['current_actual_upb']
    test.drop_column('current_actual_upb')
    test['upb_12'] = test['upb_12'].fillna(999999999)
    test['delinquency_12'] = test['delinquency_12'].fillna(-1)
    
    joined_df = test.merge(ever_df, how='left', on=['loan_id'], type='hash')
    del(ever_df)
    del(test)
    
    joined_df['ever_30'] = joined_df['ever_30'].fillna(-1)
    joined_df['ever_90'] = joined_df['ever_90'].fillna(-1)
    joined_df['ever_180'] = joined_df['ever_180'].fillna(-1)
    joined_df['delinquency_30'] = joined_df['delinquency_30'].fillna(-1)
    joined_df['delinquency_90'] = joined_df['delinquency_90'].fillna(-1)
    joined_df['delinquency_180'] = joined_df['delinquency_180'].fillna(-1)
    
    joined_df['timestamp_year'] = joined_df['timestamp_year'].astype('int32')
    joined_df['timestamp_month'] = joined_df['timestamp_month'].astype('int32')
    
    return joined_df

def cudf_combine_joined_12_mon(joined_df, testdf, **kwargs):
    joined_df.drop_column('delinquency_12')
    joined_df.drop_column('upb_12')
    joined_df['timestamp_year'] = joined_df['timestamp_year'].astype('int16')
    joined_df['timestamp_month'] = joined_df['timestamp_month'].astype('int8')
    df = joined_df.merge(testdf, how='left', on=['loan_id', 'timestamp_year', 'timestamp_month'], type='hash')
    return df

def cudf_final_performance_delinquency(df, joined_df, **kwargs):
    merged_df = cudf_null_workaround(df)
    joined_df = cudf_null_workaround(joined_df)
    merged_df['timestamp_month'] = merged_df['monthly_reporting_period'].dt.month
    merged_df['timestamp_month'] = merged_df['timestamp_month'].astype('int8')
    merged_df['timestamp_year'] = merged_df['monthly_reporting_period'].dt.year
    merged_df['timestamp_year'] = merged_df['timestamp_year'].astype('int16')
    merged_df = merged_df.merge(joined_df, how='left', on=['loan_id', 'timestamp_year', 'timestamp_month'], type='hash')
    merged_df.drop_column('timestamp_year')
    merged_df.drop_column('timestamp_month')
    return merged_df

def cudf_join_perf_acq_gdfs(perf_df, acq_df, **kwargs):
    perf_df = cudf_null_workaround(perf_df)
    acq_df = cudf_null_workaround(acq_df)
    merge_df = perf_df.merge(acq_df, how='left', on=['loan_id'], type='hash')
    return merge_df

def cudf_last_mile_cleaning(df, **kwargs):
    drop_list = [
        'loan_id', 'orig_date', 'first_pay_date', 'seller_name',
        'monthly_reporting_period', 'last_paid_installment_date', 'maturity_date', 'ever_30', 'ever_90', 'ever_180',
        'delinquency_30', 'delinquency_90', 'delinquency_180', 'upb_12',
        'zero_balance_effective_date','foreclosed_after', 'disposition_date','timestamp'
    ]
    for column in drop_list:
        df.drop_column(column)
    for col, dtype in df.dtypes.iteritems():
        if str(dtype)=='category':
            df[col] = df[col].cat.codes
        df[col] = df[col].astype('float32')
    df['delinquency_12'] = df['delinquency_12'] > 0
    df['delinquency_12'] = df['delinquency_12'].fillna(False).astype('int32')
    return df

In [ ]:

def cudf_process_quarter(names_df, acq_df, perf_df):
    acq_df = acq_df.merge(names_df, how='left', on=['seller_name'], type='hash')
    acq_df.drop_column('seller_name')
    acq_df['seller_name'] = acq_df['new']
    acq_df.drop_column('new')
    
    df = perf_df
    ever_df = cudf_create_ever_features(df)
    delinq_merge = cudf_create_delinq_features(df)
    ever_df = cudf_join_ever_delinq_features(ever_df, delinq_merge)
    del(delinq_merge)
    
    joined_df = cudf_create_joined_df(df, ever_df)
    test_df = cudf_create_12_mon_features(joined_df)
    joined_df = cudf_combine_joined_12_mon(joined_df, test_df)
    del(test_df)

    perf_df = cudf_final_performance_delinquency(df, joined_df)
    del(df, joined_df)

    final_df = cudf_join_perf_acq_gdfs(perf_df, acq_df)
    del(perf_df)
    del(acq_df)

    final_df = cudf_last_mile_cleaning(final_df)
    return final_df

def cudf_process_quarter_file(acquisition_file, perf_file):
    #Load Data
    names_df = cudf_load_names(COL_NAMES_PATH)
    acq_df = cudf_load_acquisition_csv(acquisition_file)
    perf_df = cudf_load_performance_csv(perf_file)

    df = cudf_process_quarter(names_df, acq_df, perf_df)
    return df

def process_quarter_file_multigpu(acquisition_file, perf_file, gpuid=0):
    print("On GPU {0}, Processing file: {1}".format(gpuid, perf_file))
    cuda.select_device(int(gpuid))
    
    #Load Data
    df = cudf_process_quarter_file(acquisition_file, perf_file)
    return df.to_pandas()

<a id='cudf_gpu_model_training'></a>
### Model Training (Multi-GPU)
GPU accelerated XGBoost using the *train* method

In [ ]:
import glob
from glob import glob
import os
import concurrent.futures
import numba.cuda as cuda

def process_quarters_multigpu(start_year, end_year, ngpus=len(cuda.gpus)):
    #Read the csv file(s)
    future_files = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=ngpus) as executor:
        i = 0
        gpuid = 0
        quarter = 1
        year = start_year
        while year != end_year:
            acquisition_file = ACQ_DATA_PATH + "/Acquisition_" + str(year) + "Q" + str(quarter) + ".txt"
            files = glob(os.path.join(PERF_DATA_PATH + "/Performance_" + str(year) + "Q" + str(quarter) + "*"))
            for file in files:
                future_file = executor.submit(process_quarter_file_multigpu,acquisition_file, file, gpuid)
                future_files[future_file] = file
                i = i + 1
                #Creates a gpuid from range 0:NUM_GPUS-1
                gpuid = i % ngpus

            quarter += 1
            if quarter == 5:
                year += 1
                quarter = 1

    frames = []
    for future in concurrent.futures.as_completed(future_files):
        file = future_files[future]
        try:
            df1 = future.result()
        except Exception as exc:
            print('{0} generated an exception: {1}'.format(file, exc))
        else:
            #print('{0} file is {1} bytes'.format(file, len(df1)))
            frames.append(df1)

    print("Concatenating dataframes")
    df = pd.concat(frames)    
    del frames
    
    return df

In [ ]:
%%time

#read in data files using multi-gpu
df_train = process_quarters_multigpu(START_YEAR, END_YEAR)

#Optional: Review the data and data types
#print (gdf_train.head())
#print(gdf_train.dtypes)

In [ ]:
%%time
#filter the data by the desired columns

PRED_VARS_X = gdf_train.columns.difference(['delinquency_12'])
PRED_VARS_Y ='delinquency_12'

x = df_train[PRED_VARS_X]
y = df_train[PRED_VARS_Y]


In [ ]:
rows_train = int(int(len(df_train)) * PERCENT_TRAIN)
print(rows_train)

In [ ]:
%%time 

x_train = x[:rows_train:PART_COUNT]
x_test = x[rows_train::PART_COUNT]
y_train = y[:rows_train:PART_COUNT]
y_test = y[rows_train::PART_COUNT]

In [ ]:
%%time
##Basic:
#dtrain = xgb.DMatrix(gx_train, label=gy_train)
#dtest = xgb.DMatrix(gx_test)
##Intermediate: This code executes around 1.8x times faster than basic
dtrain = xgb.DMatrix(x_train.values, label=y_train.values, nthread=-1)
dtest = xgb.DMatrix(x_test.values, nthread=-1)

In [ ]:
%%time

#To use all GPUs, set NGPUS=-1
#Optional: loop over number of GPUs for sizing and performance
NGPUS=-1

params={}
params['learning_rate'] = 0.1
params['n_estimators'] = NUM_ROUNDS
params['gamma'] = 1
params['eta'] = 0.1
params['alpha'] = 0.9
params['subsample'] = 1
params['reg_lambda'] = 1
params['scale_pos_weight'] = 2
params['min_child_weight'] = 30
params['max_depth'] = 8
params['min_child_weight'] = 3
params['early_stopping_rounds'] = 5
params['max_features'] = 'auto'
params['max_leaves'] = 2**8
params['objective'] = 'gpu:reg:linear'
params['booster'] = 'gbtree'
params['criterion'] = 'friedman_mse'
params['grow_policy'] = 'lossguide'
params['tree_method'] = 'gpu_hist'
params['n_gpus'] = NGPUS
params['silent'] = True

#evallist = [(dtest, 'validation'),(dtrain, 'train')]
evallist = [(dtrain, 'x_train')]
gbm = xgb.train(params, dtrain, NUM_ROUNDS, evallist)


### Evaluation
Accuracy (AUC) is measured by the area under the ROC curve. The definition of an acceptable AUC is relative and not absolute based on the data and feature. An area of 1 represents a perfect test; an area of .5 represents a poor or unrealiable test. A suggested rough guide for classifying the accuracy of a XGBoost:<br>
>90 - 100 = Excellent <br>
>80 - 90 = Good <br>
>70 - 80 = Fair <br>
>60 - 70 = Poor <br>
>50 - 60 = Fail <br>
>50 and Below = Unreliable

In [ ]:
from sklearn import metrics

pred = gbm.predict(dtest, ntree_limit=NUM_ROUNDS)

fpr, tpr, thresholds = metrics.roc_curve(y_test, pred)
auc = metrics.auc(fpr, tpr) * 100

acc = metrics.roc_auc_score(y_test, pred) * 100

print("AUC: {0}, ROC AUC: {1}".format(auc, acc))

In [ ]:
#cleanup for the next iteration
del acc
del auc
del fpr
del tpr
del thresholds
del pred

del gbm

gc.collect()

For additional iterations, goto top of [model training section](#cudf_gpu_model_training).

In [ ]:
#After looping through the gpus, cleanup for the benchmark
del dtrain
del dtest

del x
del y

del x_train
del x_test 
del y_train 
del y_test 

#cuDF Dataframe
del df_train

gc.collect()

<a id='cleanup'></a>
## Cleanup / Finalize

In [ ]:
gc.collect()